<a href="https://colab.research.google.com/github/takyaC/Finance/blob/main/EstimateAndActual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yahoo_fin

In [ ]:
pip install pandas

In [ ]:
pip install requests_html

In [47]:
from yahoo_fin.stock_info import get_analysts_info, get_cash_flow, get_income_statement, get_earnings_history, get_earnings, get_next_earnings_date
import pandas

In [111]:
tickers = [
#  "AAPL","AFRM","AMZN","ANTM","APPS","BMY","BNTX","CCL","CRWD","DAL",
 "DKNG","DOCU","EBS","FUTU","FVRR","GE","GOOGL","IAC","JPM","MRNA",
 "MSFT","NIO","ORCL","PINS","PLUG","PTON","RBLX","RKT","ROKU","ROOT",
 "RPRX","SBLK","SQ","TSLA","U","WKHS","WORK","ZI","ZM"
 ]

In [112]:
data_of_ticker = []

for ticker in tickers:
    try:
      # アナリスト予想（予想EPS、予想売上）
      ai = get_analysts_info(ticker)
      earnings_estimate = ai["Earnings Estimate"].iloc[[1], [1, 3, 4]]
      revenue_estimate = ai["Revenue Estimate"].iloc[[1], [1, 3, 4]]

      # EPS履歴
      earings = get_earnings(ticker)
      quarterly_results_row = len(earings["quarterly_results"]) -1
      earings_last_results = earings["quarterly_results"].iloc[quarterly_results_row, 2] # 右下のデータ(最終？Q、実績)を取得      

      # 損益計算書（四半期売上）
      ist = get_income_statement(ticker, False)
      total_revenue = ist.loc[["totalRevenue"], ist.columns[0]]

      # 単位付与(100万/１０億)
      total_revenue_by_million = total_revenue.values / 1000 / 1000
      if total_revenue_by_million < 1000:
        converted_total_revenue = "".join(str(rount(total_revenue_by_million, 2)))[1:-1] + "M"
      else:
        converted_total_revenue = "".join(str(round(total_revenue_by_million / 1000), 2))[1:-1] + "B"
      
      # 損益計算書（年間売上）
      isty = get_income_statement(ticker)
      total_revenue_yearly = isty.loc[["totalRevenue"], isty.columns[0]]

      # 単位付与(100万/１０億)
      total_revenue_yearly_by_million = total_revenue_yearly.values / 1000 / 1000
      if total_revenue_yearly_by_million < 1000:
        converted_total_revenue_yearly_by_million = "".join(str(round(total_revenue_yearly_by_million, 2)))[1:-1] + "M"
      else:
        converted_total_revenue_yearly_by_million = "".join(str(round(total_revenue_yearly_by_million / 1000), 2)[1:-1] + "B"

      # キャッシュフロー計算書（純利益、営業キャッシュフロー、リスク（純利益<営業キャッシュフローが正常)）
      cfy = get_cash_flow(ticker)
      net_income_yearly = cfy.loc[["netIncome"], cfy.columns[0]]
      operating_cash_flow_yearly = cfy.loc[["totalCashFromOperatingActivities"], cfy.columns[0]]
      is_risk_cleard  = "OK" if operating_cash_flow_yearly.values > net_income_yearly.values else "NG"

      # 営業キャッシュフローマージン (15%以上推奨)
      cash_flow_margin_yearly_before = (operating_cash_flow_yearly.values / total_revenue_yearly.values)
      cash_flow_margin_yearly = round(cash_flow_margin_yearly_before[0], 2)

      # 次回決算発表日
      next_earnings_date = get_next_earnings_date(ticker)

      # 設定
      data = []
      data.extend([ticker])

      data.extend([earings_last_results]) # 前四半期 EPS
      data.extend([converted_total_revenue]) # 前四半期 売上
      data.extend([earnings_estimate.values[0][0]]) # 今四半期予想 EPS
      data.extend([revenue_estimate.values[0][0]]) # 今四半期予想 売上
      data.extend([earnings_estimate.values[0][1]]) # 今年度予想 EPS
      data.extend([revenue_estimate.values[0][1]]) # 今年度予想 売上
      data.extend([earnings_estimate.values[0][2]]) # 次年度予想 EPS
      data.extend([revenue_estimate.values[0][2]]) # 次年度予想 売上
      # data.extend([converted_total_revenue_yearly_by_million]) # 前年度売上

      data.extend([is_risk_cleard])
      data.extend([cash_flow_margin_yearly])
      
      data.extend(["-"]) # 空列
      data.extend([next_earnings_date]) # 次回決算発表日
      data_of_ticker.append(data)

      print(data)
    except Exception as e:
      print(e)
      data_of_ticker.append([ticker])
      

['DKNG', -0.47, '322.223M', -0.4, '227.36M', -1.53, '1.01B', -1.1, '1.42B', 'OK', -0.55, '-', datetime.datetime(2021, 2, 26, 0, 0)]
['DOCU', 0.22, '430.898M', 0.22, '410.03M', 1.14, '1.87B', 1.78, '2.39B', 'OK', 0.2, '-', datetime.datetime(2021, 6, 2, 0, 0)]
['EBS', 3.19, '583.0M', 1.3, '374.1M', 9.27, '2.02B', 5.44, '1.6B', 'OK', 0.34, '-', datetime.datetime(2021, 4, 28, 0, 0)]
['FUTU', 0.1, '898.815M', 0.0, '96.73M', 1.19, '400.13M', 2.38, '685.45M', 'OK', 2.03, '-', datetime.datetime(2020, 5, 14, 0, 0)]
['FVRR', 0.12, '55.885M', -0.12, '65.15M', 0.38, '288.24M', 1.19, '384.26M', 'OK', 0.09, '-', datetime.datetime(2021, 2, 18, 0, 0)]
['GE', 0.09, '21.929B', 0.02, '17.86B', 0.26, '79.78B', 0.51, '84.58B', 'NG', 0.05, '-', datetime.datetime(2021, 4, 27, 0, 0)]
['GOOGL', 15.9, '56.898B', 15.85, '51.39B', 69.59, '226.17B', 81.06, '263.9B', 'OK', 0.36, '-', datetime.datetime(2021, 4, 26, 0, 0)]
['IAC', -0.67, '848.819M', -0.64, '820.7M', -1.33, '3.56B', 0.09, '4.17B', 'NG', 0.05, '-', dat

In [113]:
pandas.DataFrame(data_of_ticker, columns=[
        "銘柄",
        "前四半期EPS",
        "前四半期売上",
        "今四半期予想　EPS", 
        "今四半期予想　売上",
        "今年度予想　EPS", 
        "今年度予想　売上",
        "次年度予想　EPS",
        "次年度予想　売上",
        "リスク",
        "営業キャッシュフローマージン",
        "-",
        "次回決算発表日"])


,銘柄,前四半期EPS,前四半期売上,今四半期予想 EPS,今四半期予想 売上,今年度予想 EPS,今年度予想 売上,次年度予想 EPS,次年度予想 売上,リスク,営業キャッシュフローマージン,-,次回決算発表日
0,DKNG,-0.47,322.223M,-0.40,227.36M,-1.53,1.01B,-1.10,1.42B,OK,-0.55,-,2021-02-26
1,DOCU,0.22,430.898M,0.22,410.03M,1.14,1.87B,1.78,2.39B,OK,0.20,-,2021-06-02
2,EBS,3.19,583.0M,1.30,374.1M,9.27,2.02B,5.44,1.6B,OK,0.34,-,2021-04-28
3,FUTU,0.10,898.815M,0.00,96.73M,1.19,400.13M,2.38,685.45M,OK,2.03,-,2020-05-14
4,FVRR,0.12,55.885M,-0.12,65.15M,0.38,288.24M,1.19,384.26M,OK,0.09,-,2021-02-18
5,GE,0.09,21.929B,0.02,17.86B,0.26,79.78B,0.51,84.58B,NG,0.05,-,2021-04-27
6,GOOGL,15.90,56.898B,15.85,51.39B,69.59,226.17B,81.06,263.9B,OK,0.36,-,2021-04-26
7,IAC,-0.67,848.819M,-0.64,820.7M,-1.33,3.56B,0.09,4.17B,NG,0.05,-,2021-02-03
8,JPM,2.62,31.113B,2.79,29.76B,10.63,116.85B,11.42,120.1B,NG,-0.78,-,2021-04-14
9,MRNA,-0.35,570.745M,3.21,2.41B,21.96,16.6B,14.93,12.95B,OK,2.52,-,2021-05-05
